<a href="https://colab.research.google.com/github/samir1120k/Machine-Learning-Project/blob/Sentiment-Analysis/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report


In [2]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Dataset/train.csv')
df.head(20)

,text,sentiment
0,"Now, I won't deny that when I purchased this o...",neg
1,"The saddest thing about this ""tribute"" is that...",neg
2,Last night I decided to watch the prequel or s...,neg
3,I have to admit that i liked the first half of...,neg
4,I was not impressed about this film especially...,neg
5,"The original movie, The Odd Couple, has some w...",pos
6,"This was a great movie with a good cast, all o...",pos
7,"I'm a sucker for a good romance, but this one ...",neg
8,"Fast-paced, funny, sexy, and spectacular. Cagn...",pos
9,"The proverb ""Never judge a book by it's cover""...",neg


In [3]:
df.shape

(25000, 2)

In [4]:
df.isnull().sum()

,0
text,0
sentiment,0


In [5]:
from re import S
import re

In [6]:
!pip install beautifulsoup4==4.12.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 2.4 MB/s eta 0:00:00
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.12.3
    Uninstalling beautifulsoup4-4.12.3:
      Successfully uninstalled beautifulsoup4-4.12.3


In [7]:
from bs4 import BeautifulSoup

In [8]:
def remove_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

In [9]:
df['text'] = df['text'].apply(remove_html_tags)

<ipython-input-8-b26a20aa9c6e>:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


In [10]:
df.head(20)

,text,sentiment
0,"Now, I won't deny that when I purchased this o...",neg
1,"The saddest thing about this ""tribute"" is that...",neg
2,Last night I decided to watch the prequel or s...,neg
3,I have to admit that i liked the first half of...,neg
4,I was not impressed about this film especially...,neg
5,"The original movie, The Odd Couple, has some w...",pos
6,"This was a great movie with a good cast, all o...",pos
7,"I'm a sucker for a good romance, but this one ...",neg
8,"Fast-paced, funny, sexy, and spectacular. Cagn...",pos
9,"The proverb ""Never judge a book by it's cover""...",neg


In [11]:
# Convert labels to numerical values
df['sentiment']= df['sentiment'].map({'pos': 1, 'neg': 0})

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['sentiment'], test_size=0.2, random_state=42)


In [12]:
# Initialize CountVectorizer
vectorizer = CountVectorizer()
vectorizer.fit(X_train)
X_train_transformed = vectorizer.transform(X_train)
X_test_vectors = vectorizer.transform(X_test)



In [13]:
# Initialize and train the model
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_transformed, y_train)


MultinomialNB()

In [14]:
# Make predictions on the test set
y_pred = nb_classifier.predict(X_test_vectors)

# Calculate accuracy and print classification report
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print(classification_report(y_test, y_pred, target_names=['Negative', 'Positive']))


Accuracy: 0.8456
              precision    recall  f1-score   support

    Negative       0.83      0.88      0.85      2506
    Positive       0.87      0.81      0.84      2494

    accuracy                           0.85      5000
   macro avg       0.85      0.85      0.85      5000
weighted avg       0.85      0.85      0.85      5000



In [15]:
import pickle

In [16]:
# Specify the filename for your PKL file
filename = 'sentiment_model.pkl'

# Open the file in write binary ('wb') mode and save the model using pickle.dump()
with open(filename, 'wb') as file:
    pickle.dump(nb_classifier, file)

In [18]:
def predict_sentiment(text):
    text_vector = vectorizer.transform([text])
    prediction = nb_classifier.predict(text_vector)
    return "Positive" if prediction == 1 else "Negative"

# Test sample
sample_text = "this move is worse than privious one"
print("Sample text prediction:", predict_sentiment(sample_text))


Sample text prediction: Negative


In [19]:
!pip install gradio
import gradio as gr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 73.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 13.0 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [20]:
with gr.Blocks(theme="compact") as demo:
    gr.Markdown("predict_sentiment")
    #chatbot_window = gr.Chatbot(label="")
    with gr.Row():
        # Corrected 'lable' to 'label'
        text_input = gr.Textbox(label="Write the Review", placeholder="Enter your sentiment", show_label=False)
        # Create a Textbox to display the output
        output_box = gr.Textbox(label="Sentiment Prediction")
        # Submit the input to the function and display the output in the output_box
        text_input.submit(fn=predict_sentiment, inputs=text_input, outputs=output_box)
demo.launch(share=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/blocks.py:1024: UserWarning: Cannot load compact. Caught Exception: 404 Client Error: Not Found for url: https://huggingface.co/api/spaces/compact (Request ID: Root=1-6749d354-27ccb4091dab3c5f51b760cf;84f45f16-bd08-4223-b7d5-0fc8d15c8dcc)

Sorry, we can't find the page you are looking for.
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://af62717e7501a482fc.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
